
**IMPORT** **DATA**
---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

my_data = pd.read_csv("data.csv") #, nrows = 50000

#my_data

In [ ]:
import pandas as pd

cp_data = pd.read_csv("data.csv") #, nrows = 50000

#cp_data

In [ ]:
mrg_data = pd.merge(left=my_data, right=cp_data, left_on='scrip_id', right_on='id')

mrg_data.drop('id_y', axis='columns', inplace=True)
mrg_data.drop('zerodha_id', axis='columns', inplace=True)
mrg_data.drop('exchange', axis='columns', inplace=True)

mrg_data = mrg_data.rename(columns={'id_x': 'id'})

#mrg_data

In [ ]:
RL_df = mrg_data[mrg_data['scrip_id'] == 5]  
Time = RL_df
RL_df.drop('scrip_id', axis='columns', inplace=True)
RL_df.drop('name', axis='columns', inplace=True)
RL_df.drop('id', axis='columns', inplace=True)

RL_df = RL_df.rename(columns={'open_price': 'Open'})
RL_df = RL_df.rename(columns={'high_price': 'High'})
RL_df = RL_df.rename(columns={'low_price': 'Low'})
RL_df = RL_df.rename(columns={'close_price': 'Close'})
RL_df = RL_df.rename(columns={'volume': 'Volume'})
RL_df = RL_df.rename(columns={'timestamp': 'Date'})
RL_df.reset_index(inplace = True,drop=True)
#RL_df= RL_df.set_index('Date')

#RL_df.set_index('Date')

RL_df = RL_df[(RL_df['Date'] > '2020-02-24 09:15:00') & (RL_df['Date'] <= '2020-03-24 09:15:00')]
RL_df= RL_df.set_index('Date')

# start_date = '2020-02-24 09:15:00+05:30'
# end_date = '2020-03-24 09:15:00+05:30'

# mask = (RL_df['Date'] > start_date) & (RL_df['Date'] <= end_date)

# RL_df = RL_df.loc[mask]

#RL_df=RL_df.head(300)
RL_df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Open,High,Low,Close,Volume
Date,,,,,
2020-02-24 09:15:00+05:30,1469.75,1469.75,1463.10,1465.05,150673
2020-02-24 09:16:00+05:30,1464.80,1466.90,1463.85,1465.75,70326
2020-02-24 09:17:00+05:30,1465.55,1466.15,1464.00,1464.20,48558
2020-02-24 09:18:00+05:30,1464.20,1465.85,1464.05,1464.45,30801
2020-02-24 09:19:00+05:30,1464.95,1465.80,1463.80,1465.60,32519
...,...,...,...,...,...
2020-03-23 15:25:00+05:30,889.45,890.00,889.25,889.95,97520
2020-03-23 15:26:00+05:30,890.00,891.20,889.40,891.15,83728
2020-03-23 15:27:00+05:30,891.20,894.40,890.50,893.85,99600



### **PLOT DATA**

In [ ]:
# import shrimpy
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

RL_df.index = pd.to_datetime(RL_df.index).strftime("%d-%m-%Y %H:%M:%S")


RL_df['EMA_5'] = RL_df['Close'].ewm(span=5, min_periods=5).mean().shift()
RL_df['EMA_35'] = RL_df['Close'].ewm(span=35, min_periods=35).mean().shift()

RL_df['MACD'] = RL_df['EMA_5'] - RL_df['EMA_35']
RL_df['MACD_signal'] = RL_df['MACD'].ewm(span=5, min_periods=5).mean().shift()

RL_df['volumn_macd'] = RL_df['MACD'] - RL_df['MACD_signal']


import pandas as pd
from sklearn import preprocessing

# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=1,
    column_widths=[0.4],
    row_heights=[5, 5], #, 5
    vertical_spacing=0.20,
    specs=[[{"type": "Candlestick"}],
           [{"type": "Scatter"}]])
           #[{"type": "bar"}]


#ploting
from plotly.subplots import make_subplots

#fig = make_subplots(rows=2, cols=1)
fig.add_trace( go.Candlestick(x=RL_df.index.to_series(),
                                     open=RL_df.Open, 
                                     high=RL_df.High,
                                     low=RL_df.Low,
                                     close=RL_df.Close),row=1, col=1)
fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.add_trace(go.Scatter(x=RL_df.index.to_series(), y=RL_df.MACD, name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=RL_df.index.to_series(), y=RL_df.MACD_signal, name='MACD_signal'), row=2, col=1)
# fig.update_xaxes(showticklabels=False, row=2, col=1)
# fig.add_trace(go.Bar(x=RL_df.index.to_series(), y=RL_df.volumn_macd, name='Volume_macd', marker_color='red'), row=3, col=1)


fig.add_shape(
        # Line Horizontal
            type="line",
            x0=0,
            y0=0,
            x1=RL_df.shape[0],
            y1=0,
            line=dict(
                color="white",
                width=1,
                dash="dashdot",
            ),
            row=2, col=1
    )

# Rotate x-axis labels
fig.update_xaxes(tickangle=90)

# Set theme, margin, and annotation in layout
fig.update_layout(
    template="plotly_dark",
    margin=dict(r=10, t=25, b=40, l=60),
    annotations=[
        dict(
            text="Source: NOAA",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0)
    ]
)

fig.update_layout(height=600, width=1400, title_text="🎧🎵🎵🎶")

fig.show()


In [ ]:
RL_df = RL_df.reset_index()
#RL_df = RL_df.dropna()
RL_df['MACD'] = RL_df['MACD'].round(decimals=1)
RL_df['MACD_signal'] = RL_df['MACD_signal'].round(decimals=1)
RL_df

,Date,Open,High,Low,Close,Volume,EMA_5,EMA_35,MACD,MACD_signal,volumn_macd
0,24-02-2020 09:15:00,1469.75,1469.75,1463.10,1465.05,150673,NaN,NaN,NaN,NaN,NaN
1,24-02-2020 09:16:00,1464.80,1466.90,1463.85,1465.75,70326,NaN,NaN,NaN,NaN,NaN
2,24-02-2020 09:17:00,1465.55,1466.15,1464.00,1464.20,48558,NaN,NaN,NaN,NaN,NaN
3,24-02-2020 09:18:00,1464.20,1465.85,1464.05,1464.45,30801,NaN,NaN,NaN,NaN,NaN
4,24-02-2020 09:19:00,1464.95,1465.80,1463.80,1465.60,32519,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7377,23-03-2020 15:25:00,889.45,890.00,889.25,889.95,97520,888.740913,884.813583,3.9,2.7,1.237040
7378,23-03-2020 15:26:00,890.00,891.20,889.40,891.15,83728,889.143942,885.098940,4.0,3.1,0.942366
7379,23-03-2020 15:27:00,891.20,894.40,890.50,893.85,99600,889.812628,885.435110,4.4,3.4,0.960760
7380,23-03-2020 15:28:00,893.15,894.95,893.00,893.00,66096,891.158419,885.902603,5.3,3.7,1.518803


In [ ]:
RL_df = RL_df.loc[(RL_df['MACD'] == RL_df['MACD_signal'])] # | (RL_df['SMA_20'] != 0)
RL_df = RL_df.drop_duplicates(subset=['MACD'])

RL_df.reset_index(inplace = True,drop=True)
RL_df["do_this"] = ""
RL_df["qt_stock"] = ""

RL_df

,Date,Open,High,Low,Close,Volume,EMA_5,EMA_35,MACD,MACD_signal,volumn_macd,do_this,qt_stock
0,24-02-2020 09:55:00,1467.25,1468.40,1467.25,1468.20,15972,1467.713970,1467.306336,0.4,0.4,-0.029584,,
1,24-02-2020 10:06:00,1469.40,1469.40,1468.35,1468.35,18975,1469.701900,1468.285918,1.4,1.4,0.056331,,
2,24-02-2020 10:18:00,1469.85,1469.95,1469.75,1469.80,9881,1469.982384,1468.979440,1.0,1.0,-0.039992,,
3,24-02-2020 10:33:00,1468.50,1468.50,1467.50,1468.35,14378,1468.536414,1469.013789,-0.5,-0.5,-0.027084,,
4,24-02-2020 10:48:00,1466.00,1466.70,1465.55,1466.20,10002,1466.261040,1467.817129,-1.6,-1.6,0.074636,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,20-03-2020 11:34:00,974.60,976.00,974.05,975.45,49207,973.506012,966.092996,7.4,7.4,-0.022268,,
102,23-03-2020 11:27:00,896.55,897.70,895.05,897.70,54004,898.824088,906.556262,-7.7,-7.7,0.010248,,
103,23-03-2020 12:41:00,895.15,898.30,893.35,898.00,49900,897.465983,891.286277,6.2,6.2,-0.022603,,
104,23-03-2020 13:25:00,890.70,891.50,890.00,890.25,55310,893.040058,896.338652,-3.3,-3.3,0.001136,,


In [ ]:
for ix, row in RL_df.iterrows():
    if (RL_df.loc[ix, "MACD"] < -1):
        RL_df.loc[ix, "do_this"] = 'buy'
        #RL_df.loc[ix, "qt_stock"] = 1

    elif (RL_df.loc[ix, "MACD"] > 1):
        RL_df.loc[ix, "do_this"] = 'sell'
        #RL_df.loc[ix, "qt_stock"] = -1

In [ ]:
RL_df

,Date,Open,High,Low,Close,Volume,EMA_5,EMA_35,MACD,MACD_signal,volumn_macd,do_this,qt_stock
0,24-02-2020 09:55:00,1467.25,1468.40,1467.25,1468.20,15972,1467.713970,1467.306336,0.4,0.4,-0.029584,,
1,24-02-2020 10:06:00,1469.40,1469.40,1468.35,1468.35,18975,1469.701900,1468.285918,1.4,1.4,0.056331,sell,
2,24-02-2020 10:18:00,1469.85,1469.95,1469.75,1469.80,9881,1469.982384,1468.979440,1.0,1.0,-0.039992,,
3,24-02-2020 10:33:00,1468.50,1468.50,1467.50,1468.35,14378,1468.536414,1469.013789,-0.5,-0.5,-0.027084,,
4,24-02-2020 10:48:00,1466.00,1466.70,1465.55,1466.20,10002,1466.261040,1467.817129,-1.6,-1.6,0.074636,buy,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,20-03-2020 11:34:00,974.60,976.00,974.05,975.45,49207,973.506012,966.092996,7.4,7.4,-0.022268,sell,
102,23-03-2020 11:27:00,896.55,897.70,895.05,897.70,54004,898.824088,906.556262,-7.7,-7.7,0.010248,buy,
103,23-03-2020 12:41:00,895.15,898.30,893.35,898.00,49900,897.465983,891.286277,6.2,6.2,-0.022603,sell,
104,23-03-2020 13:25:00,890.70,891.50,890.00,890.25,55310,893.040058,896.338652,-3.3,-3.3,0.001136,buy,


In [ ]:
RL_df["do_this"].replace('', np.nan, inplace=True)
RL_df.dropna(subset=['do_this'], inplace=True)
RL_df

,Date,Open,High,Low,Close,Volume,EMA_5,EMA_35,MACD,MACD_signal,volumn_macd,do_this,qt_stock
1,24-02-2020 10:06:00,1469.40,1469.40,1468.35,1468.35,18975,1469.701900,1468.285918,1.4,1.4,0.056331,sell,
4,24-02-2020 10:48:00,1466.00,1466.70,1465.55,1466.20,10002,1466.261040,1467.817129,-1.6,-1.6,0.074636,buy,
9,24-02-2020 12:39:00,1474.80,1475.35,1473.75,1475.30,20566,1474.882092,1473.154188,1.7,1.7,0.052555,sell,
15,24-02-2020 13:44:00,1470.20,1472.00,1469.60,1470.60,33122,1470.245473,1471.686170,-1.4,-1.4,-0.015942,buy,
16,24-02-2020 13:53:00,1469.20,1470.15,1469.20,1470.05,9146,1469.599588,1470.901197,-1.3,-1.3,-0.016819,buy,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,20-03-2020 11:34:00,974.60,976.00,974.05,975.45,49207,973.506012,966.092996,7.4,7.4,-0.022268,sell,
102,23-03-2020 11:27:00,896.55,897.70,895.05,897.70,54004,898.824088,906.556262,-7.7,-7.7,0.010248,buy,
103,23-03-2020 12:41:00,895.15,898.30,893.35,898.00,49900,897.465983,891.286277,6.2,6.2,-0.022603,sell,
104,23-03-2020 13:25:00,890.70,891.50,890.00,890.25,55310,893.040058,896.338652,-3.3,-3.3,0.001136,buy,


In [ ]:
quant=0
for ind,row in RL_df.iterrows():
  
    if (row['do_this']=='buy'):
        if(quant==1):
            RL_df.loc[ind,"qt_stock"]=np.nan
        else:
            quant+=1
            RL_df.loc[ind,"qt_stock"]=quant
        
    elif (row['do_this']=='sell'):
        if(quant==-1):
            RL_df.loc[ind,"qt_stock"]=np.nan
        else:
            quant-=1
            RL_df.loc[ind,"qt_stock"]=quant
       
    elif (row['do_this']=='stay'):
        RL_df.loc[ind,"qt_stock"]=quant

In [ ]:
RL_df

,Date,Open,High,Low,Close,Volume,EMA_5,EMA_35,MACD,MACD_signal,volumn_macd,do_this,qt_stock
1,24-02-2020 10:06:00,1469.40,1469.40,1468.35,1468.35,18975,1469.701900,1468.285918,1.4,1.4,0.056331,sell,-1
4,24-02-2020 10:48:00,1466.00,1466.70,1465.55,1466.20,10002,1466.261040,1467.817129,-1.6,-1.6,0.074636,buy,0
9,24-02-2020 12:39:00,1474.80,1475.35,1473.75,1475.30,20566,1474.882092,1473.154188,1.7,1.7,0.052555,sell,-1
15,24-02-2020 13:44:00,1470.20,1472.00,1469.60,1470.60,33122,1470.245473,1471.686170,-1.4,-1.4,-0.015942,buy,0
16,24-02-2020 13:53:00,1469.20,1470.15,1469.20,1470.05,9146,1469.599588,1470.901197,-1.3,-1.3,-0.016819,buy,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,20-03-2020 11:34:00,974.60,976.00,974.05,975.45,49207,973.506012,966.092996,7.4,7.4,-0.022268,sell,-1
102,23-03-2020 11:27:00,896.55,897.70,895.05,897.70,54004,898.824088,906.556262,-7.7,-7.7,0.010248,buy,0
103,23-03-2020 12:41:00,895.15,898.30,893.35,898.00,49900,897.465983,891.286277,6.2,6.2,-0.022603,sell,-1
104,23-03-2020 13:25:00,890.70,891.50,890.00,890.25,55310,893.040058,896.338652,-3.3,-3.3,0.001136,buy,0


In [ ]:
RL_df= RL_df.dropna(how='any',axis=0) 
RL_df.head(2)

,Date,Open,High,Low,Close,Volume,EMA_5,EMA_35,MACD,MACD_signal,volumn_macd,do_this,qt_stock
1,24-02-2020 10:06:00,1469.4,1469.4,1468.35,1468.35,18975,1469.70190,1468.285918,1.4,1.4,0.056331,sell,-1
4,24-02-2020 10:48:00,1466.0,1466.7,1465.55,1466.20,10002,1466.26104,1467.817129,-1.6,-1.6,0.074636,buy,0


In [ ]:
transaction_df = pd.DataFrame(columns=
                              [
                               'scrip',
                               'timestamp',
                               'buy/sell',
                               'price',
                               'brokerage',
                               'qtystock',
                               'profit',
                               'cum_profit',
                               'pab',
                               'cum_pab',
                               'profit_percentage',
                               'slope'
                              ]
                             )

In [ ]:
transaction_df.drop('profit_percentage', axis='columns', inplace=True)
transaction_df.drop('slope', axis='columns', inplace=True)
transaction_df.drop('scrip', axis='columns', inplace=True)
transaction_df['timestamp'] = RL_df['Date']
transaction_df['price'] = RL_df['Close']
transaction_df['buy/sell'] = RL_df['do_this']
transaction_df['qtystock'] = RL_df['qt_stock'] 

In [ ]:
transaction_df.reset_index(inplace = True,drop=True)
transaction_df.head(10)

,timestamp,buy/sell,price,brokerage,qtystock,profit,cum_profit,pab,cum_pab
0,24-02-2020 10:06:00,sell,1468.35,NaN,-1,NaN,NaN,NaN,NaN
1,24-02-2020 10:48:00,buy,1466.20,NaN,0,NaN,NaN,NaN,NaN
2,24-02-2020 12:39:00,sell,1475.30,NaN,-1,NaN,NaN,NaN,NaN
3,24-02-2020 13:44:00,buy,1470.60,NaN,0,NaN,NaN,NaN,NaN
4,24-02-2020 13:53:00,buy,1470.05,NaN,1,NaN,NaN,NaN,NaN
5,25-02-2020 10:17:00,sell,1447.45,NaN,0,NaN,NaN,NaN,NaN
6,25-02-2020 11:02:00,buy,1437.75,NaN,1,NaN,NaN,NaN,NaN
7,25-02-2020 15:29:00,sell,1421.10,NaN,0,NaN,NaN,NaN,NaN
8,26-02-2020 09:39:00,buy,1398.20,NaN,1,NaN,NaN,NaN,NaN
9,26-02-2020 11:29:00,sell,1403.55,NaN,0,NaN,NaN,NaN,NaN


In [ ]:
import json
brokerage_file = 'BROKERAGES.json'
with open(brokerage_file) as source:
    BROKERAGES = json.load(source)

In [ ]:
BROKERAGES

{'zerodha': {'intraday': {'buy': {'base': 0.0003,
    'exchange txn charges': 3.25e-05,
    'gst': 6e-05,
    'sebi': 5e-07,
    'stamp duty': 3e-05,
    'stt': 0},
   'sell': {'base': 0.0003,
    'exchange txn charges': 3.25e-05,
    'gst': 6e-05,
    'sebi': 5e-07,
    'stamp duty': 0,
    'stt': 0.00025}}}}

In [ ]:
def calculate_brokerage(side, txn_type, price, broker):
    current_brokerages_dict=BROKERAGES[broker][txn_type]
    brokerages = 0
    for head, value in current_brokerages_dict[side].items():
        brokerages += price*value
    return brokerages

In [ ]:
for idx, row in transaction_df.iterrows():
    transaction_df.loc[idx, 'brokerage'] = calculate_brokerage(row['buy/sell'], 'intraday', row['price'],"zerodha")

In [ ]:
transaction_df.head(5)

,timestamp,buy/sell,price,brokerage,qtystock,profit,cum_profit,pab,cum_pab
0,24-02-2020 10:06:00,sell,1468.35,0.944149,-1,NaN,NaN,NaN,NaN
1,24-02-2020 10:48:00,buy,1466.20,0.620203,0,NaN,NaN,NaN,NaN
2,24-02-2020 12:39:00,sell,1475.30,0.948618,-1,NaN,NaN,NaN,NaN
3,24-02-2020 13:44:00,buy,1470.60,0.622064,0,NaN,NaN,NaN,NaN
4,24-02-2020 13:53:00,buy,1470.05,0.621831,1,NaN,NaN,NaN,NaN


In [ ]:
for ix, row in transaction_df.iterrows():
    if(ix == 0):
        transaction_df.loc[ix, "profit"] = 0
    else:       
        if(transaction_df.loc[ix, "qtystock"] == 0 and transaction_df.loc[ix - 1, "qtystock"] == 1):
            transaction_df.loc[ix, "profit"] = transaction_df.loc[ix, "price"] - transaction_df.loc[ix - 1, "price"] 
        elif(transaction_df.loc[ix, "qtystock"] == 0 and transaction_df.loc[ix - 1, "qtystock"] == -1):
            transaction_df.loc[ix, "profit"] = transaction_df.loc[ix - 1, "price"] - transaction_df.loc[ix, "price"]

In [ ]:
transaction_df['profit'] = transaction_df['profit'].replace(np.nan, 0)
transaction_df.head(10)

,timestamp,buy/sell,price,brokerage,qtystock,profit,cum_profit,pab,cum_pab
0,24-02-2020 10:06:00,sell,1468.35,0.944149,-1,0.00,NaN,NaN,NaN
1,24-02-2020 10:48:00,buy,1466.20,0.620203,0,2.15,NaN,NaN,NaN
2,24-02-2020 12:39:00,sell,1475.30,0.948618,-1,0.00,NaN,NaN,NaN
3,24-02-2020 13:44:00,buy,1470.60,0.622064,0,4.70,NaN,NaN,NaN
4,24-02-2020 13:53:00,buy,1470.05,0.621831,1,0.00,NaN,NaN,NaN
5,25-02-2020 10:17:00,sell,1447.45,0.93071,0,-22.60,NaN,NaN,NaN
6,25-02-2020 11:02:00,buy,1437.75,0.608168,1,0.00,NaN,NaN,NaN
7,25-02-2020 15:29:00,sell,1421.10,0.913767,0,-16.65,NaN,NaN,NaN
8,26-02-2020 09:39:00,buy,1398.20,0.591439,1,0.00,NaN,NaN,NaN
9,26-02-2020 11:29:00,sell,1403.55,0.902483,0,5.35,NaN,NaN,NaN


In [ ]:
transaction_df['cum_profit'] = transaction_df['profit'].cumsum(axis = 0)
transaction_df.head(10)

,timestamp,buy/sell,price,brokerage,qtystock,profit,cum_profit,pab,cum_pab
0,24-02-2020 10:06:00,sell,1468.35,0.944149,-1,0.00,0.00,NaN,NaN
1,24-02-2020 10:48:00,buy,1466.20,0.620203,0,2.15,2.15,NaN,NaN
2,24-02-2020 12:39:00,sell,1475.30,0.948618,-1,0.00,2.15,NaN,NaN
3,24-02-2020 13:44:00,buy,1470.60,0.622064,0,4.70,6.85,NaN,NaN
4,24-02-2020 13:53:00,buy,1470.05,0.621831,1,0.00,6.85,NaN,NaN
5,25-02-2020 10:17:00,sell,1447.45,0.93071,0,-22.60,-15.75,NaN,NaN
6,25-02-2020 11:02:00,buy,1437.75,0.608168,1,0.00,-15.75,NaN,NaN
7,25-02-2020 15:29:00,sell,1421.10,0.913767,0,-16.65,-32.40,NaN,NaN
8,26-02-2020 09:39:00,buy,1398.20,0.591439,1,0.00,-32.40,NaN,NaN
9,26-02-2020 11:29:00,sell,1403.55,0.902483,0,5.35,-27.05,NaN,NaN


In [ ]:
for ix, row in transaction_df.iterrows():
     transaction_df.loc[ix, "pab"] = transaction_df.loc[ix, "profit"] - transaction_df.loc[ix, "brokerage"]


In [ ]:
transaction_df['cum_pab'] = transaction_df['pab'].cumsum(axis = 0)

In [ ]:
transaction_df.head(5)

,timestamp,buy/sell,price,brokerage,qtystock,profit,cum_profit,pab,cum_pab
0,24-02-2020 10:06:00,sell,1468.35,0.944149,-1,0.00,0.00,-0.944149,-0.944149
1,24-02-2020 10:48:00,buy,1466.20,0.620203,0,2.15,2.15,1.5298,0.585648
2,24-02-2020 12:39:00,sell,1475.30,0.948618,-1,0.00,2.15,-0.948618,-0.36297
3,24-02-2020 13:44:00,buy,1470.60,0.622064,0,4.70,6.85,4.07794,3.71497
4,24-02-2020 13:53:00,buy,1470.05,0.621831,1,0.00,6.85,-0.621831,3.09314


In [ ]:
transaction_df.head(5)

,timestamp,buy/sell,price,brokerage,qtystock,profit,cum_profit,pab,cum_pab
0,24-02-2020 10:06:00,sell,1468.35,0.944149,-1,0.00,0.00,-0.944149,-0.944149
1,24-02-2020 10:48:00,buy,1466.20,0.620203,0,2.15,2.15,1.5298,0.585648
2,24-02-2020 12:39:00,sell,1475.30,0.948618,-1,0.00,2.15,-0.948618,-0.36297
3,24-02-2020 13:44:00,buy,1470.60,0.622064,0,4.70,6.85,4.07794,3.71497
4,24-02-2020 13:53:00,buy,1470.05,0.621831,1,0.00,6.85,-0.621831,3.09314


In [ ]:
transaction_df= transaction_df.set_index('timestamp')
transaction_df.tail(10)

,buy/sell,price,brokerage,qtystock,profit,cum_profit,pab,cum_pab
timestamp,,,,,,,,
18-03-2020 09:38:00,buy,1008.15,0.426447,0,2.15,237.55,1.72355,206.341
18-03-2020 11:00:00,sell,1022.35,0.657371,-1,0.00,237.55,-0.657371,205.683
19-03-2020 12:52:00,buy,919.90,0.389118,0,102.45,340.00,102.061,307.744
19-03-2020 15:07:00,buy,921.40,0.389752,1,0.00,340.00,-0.389752,307.355
20-03-2020 11:32:00,sell,973.30,0.625832,0,51.90,391.90,51.2742,358.629
20-03-2020 11:34:00,sell,975.45,0.627214,-1,0.00,391.90,-0.627214,358.002
23-03-2020 11:27:00,buy,897.70,0.379727,0,77.75,469.65,77.3703,435.372
23-03-2020 12:41:00,sell,898.00,0.577414,-1,0.00,469.65,-0.577414,434.794
23-03-2020 13:25:00,buy,890.25,0.376576,0,7.75,477.40,7.37342,442.168


In [ ]:
fig = go.Figure(go.Bar(x=transaction_df.index.to_series(), y=transaction_df.cum_pab, name='cum_pab', marker_color='blue'))
fig.show()